In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
%%time
pd.set_option('display.width', 300)
pd.set_option('display.max_columns', 300)

data = pd.read_csv('D://Github//ML_Project//single_test//car.data', header = None)
n_columns = len(data.columns)
columns = ['buy', 'maintain', 'doors', 'persons', 'boot', 'saftey', 'accept']
new_columns = dict(list(zip(np.arange(n_columns), columns)))
data.rename(columns = new_columns, inplace = True)


Wall time: 19.9 ms


## one-hot编码

In [3]:
# one-hot编码
x = pd.DataFrame()
for col in columns[:-1]:
    t = pd.get_dummies(data[col])
    t = t.rename(columns=lambda x: col+'_'+str(x))
    x = pd.concat((x, t), axis=1)

y = pd.Categorical(data['accept']).codes

## 拟合曲线

In [4]:
x, x_test, y, y_test = train_test_split(x, y, test_size=0.3)
clf = LogisticRegressionCV()
clf.fit(x, y)
x, x_test, y, y_test = train_test_split(x, y, test_size = 0.3)    #分割训练/测试集
#使用交叉验证进行对数回归，惩罚项由lg(10^-3)~lg(10^4), 5折交叉
#clf = LogisticRegressionCV(Cs = np.logspace(-3, 4, 8), cv = 5)
clf = RandomForestClassifier(n_estimators = 50, max_depth = 7)
clf.fit(x, y)

y_hat = clf.predict(x)
print('训练集精度：', metrics.accuracy_score(y, y_hat))
y_test_hat = clf.predict(x_test)
print('测试集精度：', metrics.accuracy_score(y_test, y_test_hat))
n_class = len(np.unique(y))

训练集精度： 0.96926713948
测试集精度： 0.900826446281


In [5]:
y_test_one_hot = label_binarize(y_test, classes = np.arange(n_class))
y_test_one_hot_hat = clf.predict_proba(x_test)
fpr, tpr, _ = metrics.roc_curve(y_test_one_hot.ravel(), y_test_one_hot_hat.ravel())
print('Micro AUC:\t', metrics.auc(fpr, tpr))

Micro AUC:	 0.988472250681


In [16]:
from sklearn.preprocessing import OneHotEncoder

In [17]:
ohe = OneHotEncoder( sparse = False)
x = [[1, 2, 1],
     [1, 2, 0],
     [2, 0, 2],
     [0, 2, 2]]
x_onehot = ohe.fit_transform(x)
x_onehot

array([[ 0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.]])